# Encoder-only Transformer Architektur

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 15.0 MB/s eta 0:00:00


In [3]:
import sys
import random
import numpy as np
import pandas as pd
import ast
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import time

#sys.path.append('../scripts')
sys.path.append('/content/drive/MyDrive/PMDS/Notebooks')
import ml_helper

In [4]:
data_path = '/content/drive/MyDrive/PMDS/Data'
#data_path = '../data'

In [5]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


## Prepare Test and Training Data Loader

In [7]:
amino_acids = ['A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V', '*',
               '_']

aminoacids_to_integer = dict((a, i) for i, a in enumerate(amino_acids))
integer_to_aminoacids = dict((i, a) for i, a in enumerate(amino_acids))

codons = ['TTT', 'TTC', 'TTA', 'TTG', 'TCT', 'TCC', 'TCA', 'TCG', 'TAT', 'TAC', 'TAA', 'TAG', 'TGT', 'TGC', 'TGA',
          'TGG', 'CTT', 'CTC', 'CTA', 'CTG', 'CCT', 'CCC', 'CCA', 'CCG', 'CAT', 'CAC', 'CAA', 'CAG', 'CGT', 'CGC',
          'CGA', 'CGG', 'ATT', 'ATC', 'ATA', 'ATG', 'ACT', 'ACC', 'ACA', 'ACG', 'AAT', 'AAC', 'AAA', 'AAG', 'AGT',
          'AGC', 'AGA', 'AGG', 'GTT', 'GTC', 'GTA', 'GTG', 'GCT', 'GCC', 'GCA', 'GCG', 'GAT', 'GAC', 'GAA', 'GAG',
          'GGT', 'GGC', 'GGA', 'GGG', '___']

codons_to_integer = dict((c, i) for i, c in enumerate(codons))
integer_to_codons = dict((i, c) for i, c in enumerate(codons))

In [8]:
organism = "E.Coli"
min_length = None
max_length = 500

train_dataset = ml_helper.CodonDataset(organism, "train", min_length, max_length, cut_data=True, one_hot_aa=False, data_path=data_path, device=device)
print(f"Länge train_dataset: {len(train_dataset)}")
test_dataset = ml_helper.CodonDataset(organism, "test", min_length, max_length, cut_data=True, one_hot_aa=False, data_path=data_path, device=device)
print(f"Länge test_dataset: {len(test_dataset)}")

Länge train_dataset: 3561
Länge test_dataset: 864


In [9]:
print(train_dataset[3])
print(train_dataset[3][0].shape)
print(train_dataset[3][1].shape)

(tensor([12, 16, 18,  0, 19,  6, 13,  3,  2, 19, 15,  1, 10, 18,  7,  3, 19,  1,
         0, 19,  3,  7, 19, 15,  9,  0,  9, 11,  3,  7,  6, 13, 13, 15, 12, 10,
         7, 14, 15,  7, 15,  7, 11, 16, 16,  4, 10,  1, 10,  9,  0,  7, 13,  6,
         5, 10, 15,  7,  7,  0,  9, 15,  9, 13,  7, 11, 14,  0, 15,  2, 10, 14,
        14, 17,  6,  1,  3, 19,  2, 16, 19, 13,  5,  3, 18,  0, 10, 13, 14,  8,
        12, 15,  9, 10,  3,  2, 19,  0, 18,  7, 10, 12, 19, 11,  7, 19,  2, 11,
        11,  5,  1,  8,  0, 12,  0,  5,  6,  0, 10,  6, 11, 19,  0, 10,  7, 13,
        19,  8,  5,  1, 11, 14, 15,  5, 10, 15,  7,  7,  5,  1,  5,  1, 19,  0,
         9,  0,  1,  0, 10, 19,  2,  6, 14,  1, 19, 10, 10, 10,  3,  6, 14, 10,
         7,  0, 10,  3, 10, 11, 10,  1,  6,  5, 12,  5, 10,  6, 10, 11, 11, 10,
         5,  5, 15, 10,  7,  9, 16, 13,  9, 13, 19, 16,  8,  3,  5,  7,  6,  0,
        10, 15, 12, 15,  3,  1, 19,  0, 19, 13,  2,  2,  7,  1,  9,  6,  5, 19,
         3, 15, 14,  1,  3, 10, 18, 12,

In [10]:
BATCH_SIZE = 32
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

## Define the encoder-only model

In [11]:
class EncoderClassifier(nn.Module):
    def __init__(self, embed_dim, num_layers, num_heads):
        super(EncoderClassifier, self).__init__()

        self.emb = nn.Embedding(len(amino_acids), embed_dim, padding_idx=len(amino_acids)-1)
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim,
            nhead=num_heads,
            batch_first=True
        )
        self.encoder = nn.TransformerEncoder(
            encoder_layer=self.encoder_layer,
            num_layers=num_layers,
        )
        self.linear = nn.Linear(embed_dim, len(codons))
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = x.long()
        x = self.emb(x)
        x = self.encoder(x)
        x = self.dropout(x)
        out = self.linear(x)
        return out

In [12]:
EMBED_DIM = 256
NUM_ENCODER_LAYERS = 3
NUM_HEADS = 4

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = EncoderClassifier(
    embed_dim=EMBED_DIM,
    num_layers=NUM_ENCODER_LAYERS,
    num_heads=NUM_HEADS
).to(device)
print(model)

EncoderClassifier(
  (emb): Embedding(22, 256, padding_idx=21)
  (encoder_layer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
    )
    (linear1): Linear(in_features=256, out_features=2048, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=2048, out_features=256, bias=True)
    (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
  )
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-2): 3 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
        )
        (linear1): Linear(in_features=256, out_features=2048, bias=True)
        (dropout

In [14]:
# Total parameters and trainable parameters.
def print_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    print(f"{total_params:,} total parameters.")
    total_trainable_params = sum(
        p.numel() for p in model.parameters() if p.requires_grad)
    print(f"{total_trainable_params:,} training parameters.")

In [15]:
print_parameters(model)

5,282,625 total parameters.
5,282,625 training parameters.


In [16]:
def test_forward_pass(model, data_loader):
  batch_data, batch_label = next(iter(data_loader))
  print(f"input dim: {batch_data.shape}")
  output = model(batch_data)
  print(f"output dim: {output.shape}")

In [17]:
test_forward_pass(model, train_loader)

input dim: torch.Size([32, 500])
output dim: torch.Size([32, 500, 65])


## Define the training methods

In [18]:
def train_model(model, optimizer, criterion, num_epochs, print_batches=0):
    for epoch in range(num_epochs):
        model.train()

        epoch_start_time = time.time()
        batch_start_time = time.time()
        epoch_loss = 0
        for batch_idx, batch in enumerate(train_loader):
            # Clear gradients
            optimizer.zero_grad()

            # Forward pass
            input_data, labels = batch

            output = model(input_data)  # (batch_size, seq_len, num_classes)
            output = output.view(-1, len(codons)) # (batch_size * seq_len, num_classes)

            labels = labels.view(-1).long() # (batch_size, seq_len) -> (batch_size * seq_len)

            # Calculate loss
            loss = criterion(output, labels)
            epoch_loss += loss.item()

            # Backward pass
            loss.backward()

            # Update model parameters
            optimizer.step()

            if print_batches != 0 and batch_idx % print_batches == (print_batches-1):
                batch_time =  round(time.time() - batch_start_time,2)
                print(f'Batch [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}, Time since last batch print: {batch_time} s')
                batch_start_time = time.time()

        epoch_time = round(time.time() - epoch_start_time,2)
        epoch_loss = round(epoch_loss / len(train_loader),4)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss}, Took {epoch_time} s')

## Define the evaluation methods to calculate metrics

In [47]:
from sklearn.metrics import accuracy_score

def compute_accuracy(predictions, labels):
    predictions = np.argmax(predictions, axis=1)

    # Find indices where labels are not equal to the padding value
    non_padding_indices = labels != codons_to_integer['___']

    # Filter out predictions and labels where the label is not padding
    filtered_predictions = predictions[non_padding_indices]
    filtered_labels = labels[non_padding_indices]

    acc = accuracy_score(labels, predictions)
    return acc

In [38]:
def evaluate_model(model, criterion):
    model.eval()  # Set the model to evaluation mode

    total_loss = 0.0

    with torch.no_grad():
        accuracies = []
        for batch_idx, batch in enumerate(test_loader):
             # Forward pass
            input_data, labels = batch

            output = model(input_data)  # (batch_size, seq_len, num_classes)
            output = output.view(-1, len(codons)) # (batch_size * seq_len, num_classes)

            labels = labels.view(-1).long() # (batch_size, seq_len) -> (batch_size * seq_len)

            # Calculate loss
            loss = criterion(output, labels)

            # Compute total loss
            total_loss += loss.item()

            # Compute custom metrics
            accuracy = compute_accuracy(output.cpu(), labels.cpu())
            accuracies.append(accuracy)

    # Compute average loss
    avg_loss = total_loss / len(test_loader)

    # Compute average accuracy
    avg_accuracy = np.mean(accuracies)

    print(f'Average Loss: {avg_loss:.4f}')
    print(f'Average Accuracy: {avg_accuracy:.4f}')

    return output, labels

## Training the model

In [32]:
EMBED_DIM = 256
NUM_ENCODER_LAYERS = 4
NUM_HEADS = 4

In [33]:
model = EncoderClassifier(
    embed_dim=EMBED_DIM,
    num_layers=NUM_ENCODER_LAYERS,
    num_heads=NUM_HEADS
).to(device)
print_parameters(model)

6,597,697 total parameters.
6,597,697 training parameters.


In [34]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [35]:
EPOCHS = 10
print("----- Start Training -----")
train_model(model, optimizer, criterion, EPOCHS)

----- Start Training -----
Epoch [1/10], Loss: 0.7352, Took 28.89 s
Epoch [2/10], Loss: 0.5723, Took 28.76 s
Epoch [3/10], Loss: 0.5681, Took 27.95 s
Epoch [4/10], Loss: 0.5636, Took 28.3 s
Epoch [5/10], Loss: 0.5621, Took 28.43 s
Epoch [6/10], Loss: 0.5599, Took 28.24 s
Epoch [7/10], Loss: 0.5607, Took 28.25 s
Epoch [8/10], Loss: 0.5596, Took 28.31 s
Epoch [9/10], Loss: 0.5611, Took 28.35 s
Epoch [10/10], Loss: 0.5595, Took 28.32 s


In [36]:
batch_data, batch_label = next(iter(train_loader))
output = model(batch_data)
print(output.shape)

torch.Size([32, 500, 65])


In [37]:
 torch.save(model.state_dict(), data_path+"/encoder_256_4_4_10e.pkl")

In [ ]:
model = EncoderClassifier(
    embed_dim=EMBED_DIM,
    num_layers=NUM_ENCODER_LAYERS,
    num_heads=NUM_HEADS
).to(device)
model.load_state_dict(torch.load(data_path+"/encoder_256_4_4_10e.pkl", map_location=device))

<All keys matched successfully>

In [48]:
evaluate_model(model, criterion)

Average Loss: 0.5520
Average Accuracy: 0.7447


(tensor([[ 0.5397,  0.3644,  0.0784,  ..., -0.2049, -0.1571, -0.3564],
         [-1.0555, -0.9793, -0.9321,  ..., -0.5827, -0.2546,  0.6944],
         [-0.3606, -0.4771, -0.6952,  ..., -0.3903, -0.7320, -0.0236],
         ...,
         [-1.6050, -1.3830, -1.3978,  ..., -2.0612, -1.8123, 12.6611],
         [-1.6050, -1.3830, -1.3978,  ..., -2.0612, -1.8123, 12.6611],
         [-1.6050, -1.3830, -1.3978,  ..., -2.0612, -1.8123, 12.6611]],
        device='cuda:0'),
 tensor([35, 52, 40,  ..., 64, 64, 64], device='cuda:0'))